# Импорт библиотек

In [2]:
import mlflow

import pandas as pd
import numpy as np

from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score

from transformers import pipeline
from transformers import Trainer, TrainingArguments


import nltk
from nltk.tokenize import word_tokenize

nltk.download('stopwords')

from nltk.corpus import stopwords
from functools import lru_cache

from pymorphy3 import MorphAnalyzer
import re
import emoji

c:\Users\Smart\PycharmProjects\NLP_and_MLOPS_course_work\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Smart\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Эксперименты с датасетами

## Первый эксперимент

In [3]:
#!   mlflow server --host 127.0.0.1 --port 8080

In [4]:
# Подключаемся к серверу, запущенным на локальном хосте

mlflow.set_tracking_uri("http://127.0.0.1:8080")

In [5]:
analyzer = MorphAnalyzer(lang = 'ru')
stop_words = nltk.corpus.stopwords.words('russian')
stop_words_cleaned = []

for i in stop_words:
    if i != 'не' and i != 'нет' and i != 'никогда':
        stop_words_cleaned.append(i)


@lru_cache(maxsize=100_000)
def lemmatization(text):
    return analyzer.parse(text)[0].normal_form

def preprocess_text(text):

    text = re.sub(r'[\n\r\t]', ' ', text)
    
    text = re.sub(r'\bn(\w+)', r'\1', text)  # nчипсы -> чипсы
    
    text = re.sub(r'[^\w\s\d]', '', text)
    
    text = re.sub(r'\s+', ' ', text).strip()



    text_tokens = nltk.word_tokenize(text)
    sentence = []

    for i in text_tokens:
        if i not in stop_words_cleaned:
            sentence.append(lemmatization(i))

    return ' '.join(sentence)


print(preprocess_text('Всем привет! Какое же неприятное место, нет?\n'))



весь привет какой неприятный место нет


In [ ]:
with mlflow.start_run(run_name='First dataset'):
    
    mlflow.set_tag('Dataset_version', '1.0.0')

    df_annotations_1 = pd.read_json("project-8-at-2025-11-01-11-43-483e368c.json")
    df_annotations_2 = pd.read_json("project-9-at-2025-11-01-11-51-0428f45f.json")
    df_annotations_3 = pd.read_json("project-10-at-2025-11-06-14-16-bb9bc9cd.json")

    df_annotations = pd.concat([df_annotations_1, df_annotations_2, df_annotations_3])

    df = pd.DataFrame(columns = ['span', 'label'])

    for mark in df_annotations['aspect_sentiment']:
        for param in range(len(mark)):
            span = mark[param]['text']
            label = mark[param]['labels'][0]

            df.loc[len(df)] = [span, label]

    df['span'] = df['span'].apply(preprocess_text)
    
    
    display(df)


    file = df.to_csv('Первая_версия.csv')
    
    
    mlflow.log_artifact('Первая_версия.csv', 'datasets')


,span,label
0,вкус реально отличный,ВКУС_POSITIVE
1,я есть чипсы очень долго год 2 назад советоват...,O
2,хотеться купить очень последний время стать оч...,O
3,вкус иметь каждый 2 пачка,O
4,вкус который стать намного острый обычный,ВКУС_NEGATIVE
...,...,...
2063,итогчипсы лейс flamin hot острый креветка васа...,ВКУС_POSITIVE
2064,у привлекательный аппетитный вид,ПАЧКА_POSITIVE
2065,оригинальный пикантный вкус,ВКУС_POSITIVE
2066,к покупка рекомендовать,ВКУС_POSITIVE


🏃 View run First dataset at: http://127.0.0.1:8080/#/experiments/0/runs/dd8fbbca4fae413eb06e3e1e7d1991d7
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/0


## Второй эксперимент

In [ ]:
def clean_text_only(text):

    if not isinstance(text, str):
        return ""
    
    text = text.lower()
    
    # Заменяем эмодзи
    text = emoji.demojize(text, delimiters=(" ", " "))
    
    # Удаляем переносы и спецсимволы
    text = re.sub(r'[\n\r\t]', ' ', text)
    
    # Удаляем пунктуацию
    text = re.sub(r'[^\w\s]', ' ', text)
    
    # Убираем лишние пробелы
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

In [ ]:
with mlflow.start_run(run_name = 'Second dataset'):
    
    mlflow.set_tag('Dataset_version', '2.0.0')

    df_annotations_1 = pd.read_json("project-8-at-2025-11-01-11-43-483e368c.json")
    df_annotations_2 = pd.read_json("project-9-at-2025-11-01-11-51-0428f45f.json")
    df_annotations_3 = pd.read_json("project-10-at-2025-11-06-14-16-bb9bc9cd.json")

    df_annotations = pd.concat([df_annotations_1, df_annotations_2, df_annotations_3])

    df = pd.DataFrame(columns = ['span', 'label'])

    for mark in df_annotations['aspect_sentiment']:
        for param in range(len(mark)):
            span = mark[param]['text']
            label = mark[param]['labels'][0]

            df.loc[len(df)] = [span, label]
    
    df['span'] = df['span'].apply(clean_text_only)

    file = df.to_csv('Вторая_версия.csv')
    
    display(df)

    mlflow.log_artifact('Вторая_версия.csv', 'datasets')

,span,label
0,вкус был реально отличный,ВКУС_POSITIVE
1,я ел эти чипсы очень долго еще года 2 назад со...,O
2,хотелось купить очень много но в последнее вре...,O
3,сейчас такой вкус имеет каждая 2 пачка,O
4,вкус которых стал намного острее обычного,ВКУС_NEGATIVE
...,...,...
2063,check_mark_button итог nчипсы лейс flamin hot ...,ВКУС_POSITIVE
2064,у них привлекательный и аппетитный вид,ПАЧКА_POSITIVE
2065,оригинальный пикантный вкус,ВКУС_POSITIVE
2066,к покупке рекомендую,ВКУС_POSITIVE


🏃 View run Second dataset at: http://127.0.0.1:8080/#/experiments/0/runs/8d1fbaf0b5b14e268b46fc2a4a0c2a4f
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/0


# Эксперементирование с моделями

## 